# PyCity Schools Analysis

*Sandra Mejia Avendaño*

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645-675) underperformed compared to schools with smaller budgets (<\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Data gathering and organizing

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.shape

(39170, 11)

In [3]:
school_data_complete = school_data_complete.rename(columns={"student_name":"Student Name", "gender":"Gender", \
                                                            "grade":"Grade", "school_name":"School Name", \
                                                            "reading_score":"Reading Score", "math_score":"Math Score", \
                                                            "type":"Type", "size":"Size", "budget":"Budget"})
school_data_complete.head()

,Student ID,Student Name,Gender,Grade,School Name,Reading Score,Math Score,School ID,Type,Size,Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


***

## District Summary
Snapshot of the district's key metrics:

* Total number of schools

* Total number of students

* Total budget

* Average math score 

* Average reading score

* Overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Percentage of students with a passing math score (70 or greater)

* Percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results


In [4]:
total_schools = len(school_data_complete["School Name"].unique())
total_students = school_data_complete["Student Name"].count()
total_budget = school_data_complete["Budget"].unique().sum()
avg_math = school_data_complete["Math Score"].mean()
avg_read = school_data_complete["Reading Score"].mean()
pass_math = 100*(school_data_complete
             .loc[school_data_complete["Math Score"]>= 70, "Math Score"].count()
             /school_data_complete["Math Score"].count()
            )
pass_read = 100*(school_data_complete
             .loc[school_data_complete["Reading Score"]>= 70, "Reading Score"].count()
             /school_data_complete["Reading Score"].count()
            )
pass_overall = (pass_math+pass_read)/2

In [5]:
#total_budget = format(total_budget, '${:,.2f}')
type(total_budget)

numpy.int64

In [6]:
global_summary = pd.DataFrame({"Total Schools":[total_schools], "Total Students": '{:,d}'.format(total_students), 
                               "Total Budget": '${:,.2f}'.format(total_budget),
                               "Average Math Score": '{:.2f}'.format(avg_math), 
                               "Average Reading Score": '{:.2f}'.format(avg_read), 
                               "% Passing Math": '{:.2f}'.format(pass_math), 
                               "% Passing Reading": '{:.2f}'.format(pass_read), 
                               "%Overall Passing Rate": '{:.2f}'.format(pass_overall)})
global_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.39


***

## School Summary

Overview table that summarizing key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
school_data_by_school = school_data_complete.groupby("School Name")

In [8]:
# Average Reading and Math Score
school_summary = pd.DataFrame (school_data_by_school["Reading Score", "Math Score"].mean())
school_summary = school_summary.rename(columns={"Reading Score":"Average Reading Score", "Math Score":"Average Math Score"})

# Percentage passing Math and Reading
school_summary["% Passing Reading"] = 100*(school_data_complete
                                    .loc[school_data_complete["Reading Score"] >= 70]
                                    .groupby("School Name")["Reading Score"].count()
                                    /school_data_by_school["Reading Score"].count()
                                   )

school_summary["% Passing Math"] = 100*(school_data_complete
                                    .loc[school_data_complete["Math Score"] >= 70]
                                    .groupby("School Name")["Math Score"].count()
                                    /school_data_by_school["Math Score"].count()
                                   )
# Overall Passing Rate
school_summary["% Overall Passing Rate"] = (school_summary["% Passing Math"] + school_summary["% Passing Reading"])/2

# Total Students
school_summary["Total Students"] = school_data_by_school["Student Name"].count()

#Total School Budget
school_summary["Total School Budget"] = school_data_by_school["Budget"].mean()


#Per Student Budget
school_summary["Per Student Budget"] = school_summary["Total School Budget"] / school_summary["Total Students"]

#School Type
school_summary["School Type"] = school_data_by_school["Type"].first()

In [9]:
# Reorganizing columns
school_summary = school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                                "% Overall Passing Rate"]]
# Formatting
school_summary["Average Math Score"] = (school_summary["Average Math Score"].astype(float).map("{:,.2f}".format))
school_summary["Average Reading Score"] = (school_summary["Average Reading Score"].map("{:,.2f}".format))
school_summary["% Passing Math"] = (school_summary["% Passing Math"].map("{:,.2f}".format))
school_summary["% Passing Reading"] = (school_summary["% Passing Reading"].map("{:,.2f}".format))
school_summary["% Overall Passing Rate"] = (school_summary["% Overall Passing Rate"].map("{:,.2f}".format))
school_summary["Per Student Budget"] = (school_summary["Per Student Budget"].map("${:,.2f}".format))
school_summary["Total School Budget"] = (school_summary["Total School Budget"].map("${:,.2f}".format))

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.38
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64


***

## Top Performing Schools (By Passing Rate)

In [10]:
top_performing = (school_summary
                  .sort_values("% Overall Passing Rate", ascending=False)
                  .iloc[0:5,:]
                 )
top_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.20


***

## Bottom Performing Schools (By Passing Rate)

In [11]:
bottom_performing = (school_summary
                  .sort_values("% Overall Passing Rate")
                  .iloc[0:5,:]
                 )
bottom_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80


***

## Reading Scores by Grade

Average **Reading** Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [12]:
# Pandas series for each grade
# Group each series by school

read_9th = (school_data_complete
         .loc[school_data_complete["Grade"]=="9th", ["School Name","Reading Score"]]
         .groupby("School Name")["Reading Score"].mean()
         .rename ("9th"))
read_10th = (school_data_complete
         .loc[school_data_complete["Grade"]=="10th", ["School Name","Reading Score"]]
         .groupby("School Name")["Reading Score"].mean()
         .rename("10th"))
read_11th = (school_data_complete
         .loc[school_data_complete["Grade"]=="11th", ["School Name","Reading Score"]]
         .groupby("School Name")["Reading Score"].mean()
         .rename("11th"))
read_12th = (school_data_complete
         .loc[school_data_complete["Grade"]=="12th", ["School Name","Reading Score"]]
         .groupby("School Name")["Reading Score"].mean()
         .rename("12th"))

In [13]:
# Combine the series into a dataframe

read = pd.concat([read_9th, read_10th, read_11th, read_12th], axis=1)
read

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


***

## Math Score by Grade 

Average **Math** Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [14]:
math_9th = (school_data_complete
         .loc[school_data_complete["Grade"]=="9th", ["School Name","Math Score"]]
         .groupby("School Name")["Math Score"].mean()
         .rename ("9th"))
math_10th = (school_data_complete
         .loc[school_data_complete["Grade"]=="10th", ["School Name","Math Score"]]
         .groupby("School Name")["Math Score"].mean()
         .rename("10th"))
math_11th = (school_data_complete
         .loc[school_data_complete["Grade"]=="11th", ["School Name","Math Score"]]
         .groupby("School Name")["Math Score"].mean()
         .rename("11th"))
math_12th = (school_data_complete
         .loc[school_data_complete["Grade"]=="12th", ["School Name","Math Score"]]
         .groupby("School Name")["Math Score"].mean()
         .rename("12th"))

In [15]:
math = pd.concat([math_9th, math_10th, math_11th, math_12th], axis=1)
math
#reading_per_grade = pd.DataFrame(read_9th, read_10th, read_11th, read_12th)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


***

## Scores by School Spending

School spending per student grouped in **5** bins.
Table breaking down school performances based on average Spending Ranges (Per Student). 
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [16]:
# Calculation of Spending Per Student
school_data_complete["Spending per Student"] = school_data_complete["Budget"]/school_data_complete["Size"]

In [17]:
print("Maximum spending per student: " + str(school_data_complete["Spending per Student"].max()))
print("Minimum spending per student: " + str(school_data_complete["Spending per Student"].min()))

Maximum spending per student: 655.0
Minimum spending per student: 578.0


In [18]:
# Bins
spending_bins = [0, 600, 630, 640, 650, 675]
group_names = ["<$600", "$600-630", "$630-640", "$640-650", "$650-675"]

In [19]:
# Binning of Spending per Student
school_data_complete["Spending Range"] = pd.cut(school_data_complete["Spending per Student"], spending_bins, labels=group_names)

In [20]:
# Number of schools in each bin
school_data_complete["Spending Range"].value_counts()

$630-640    8583
<$600       8129
$650-675    7552
$640-650    7500
$600-630    7406
Name: Spending Range, dtype: int64

**Number and size of bins was adjusted so that a similar number of schools fell into each bin**

In [21]:
school_data_by_spending = school_data_complete.groupby("Spending Range")

In [22]:
# Average Reading and Math Score
school_summary_spending = pd.DataFrame (school_data_by_spending["Math Score", "Reading Score"].mean())
school_summary_spending = school_summary_spending.rename(columns={"Math Score":"Average Math Score", 
                                                        "Reading Score":"Average Reading Score"})

school_summary_spending["% Passing Math"] = 100*(school_data_complete
                                    .loc[school_data_complete["Math Score"] >= 70]
                                    .groupby("Spending Range")["Math Score"].count()
                                    /school_data_by_spending["Math Score"].count()
                                   )

# Percentage passing Math and Reading
school_summary_spending["% Passing Reading"] = 100*(school_data_complete
                                    .loc[school_data_complete["Reading Score"] >= 70]
                                    .groupby("Spending Range")["Reading Score"].count()
                                    /school_data_by_spending["Reading Score"].count()
                                   )


# Overall Passing Rate
school_summary_spending["% Overall Passing Rate"] = (school_summary_spending["% Passing Math"] 
                                                     + school_summary_spending["% Passing Reading"])/2

school_summary_spending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range,,,,,
<$600,83.362283,83.912412,93.738467,96.506335,95.122401
$600-630,79.179989,81.976641,75.600864,86.767486,81.184175
$630-640,78.050332,81.478038,71.361995,83.653734,77.507864
$640-650,77.083467,80.886000,66.880000,80.520000,73.700000
$650-675,77.034693,81.030323,66.340042,81.038136,73.689089


***

## Scores by School Size

School size grouped in **3** bins.
Table breaking down school performances based on School Size.

In [23]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 2000, 3000, 5000]
group_names = ["Small (<2000)", "Medium (2000-3000)", "Large (3000-5000)"]

In [24]:
school_data_complete["Size Range"] = pd.cut(school_data_complete["Size"], size_bins, labels=group_names)


In [25]:
# Number of schools in each bin
school_data_complete["Size Range"].value_counts()

Large (3000-5000)     18371
Medium (2000-3000)    10888
Small (<2000)          9911
Name: Size Range, dtype: int64

In [26]:
school_data_by_size = school_data_complete.groupby("Size Range")

In [27]:
# Average Reading and Math Score
school_summary_size = pd.DataFrame (school_data_by_size["Math Score", "Reading Score"].mean())
school_summary_size = school_summary_size.rename(columns={"Math Score":"Average Math Score", 
                                                        "Reading Score":"Average Reading Score"})

school_summary_size["% Passing Math"] = 100*(school_data_complete
                                    .loc[school_data_complete["Math Score"] >= 70]
                                    .groupby("Size Range")["Math Score"].count()
                                    /school_data_by_size["Math Score"].count()
                                   )

# Percentage passing Math and Reading
school_summary_size["% Passing Reading"] = 100*(school_data_complete
                                    .loc[school_data_complete["Reading Score"] >= 70]
                                    .groupby("Size Range")["Reading Score"].count()
                                    /school_data_by_size["Reading Score"].count()
                                   )


# Overall Passing Rate
school_summary_size["% Overall Passing Rate"] = (school_summary_size["% Passing Math"] 
                                                     + school_summary_size["% Passing Reading"])/2

school_summary_size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Size Range,,,,,
Small (<2000),83.436586,83.882857,93.663606,96.670366,95.166986
Medium (2000-3000),78.164034,81.654758,72.336517,83.844600,78.090558
Large (3000-5000),77.070764,80.928365,66.468891,81.106091,73.787491


***

## Scores by School Type

Table breaking down school performances based on School Type.

In [28]:
school_data_by_type = school_data_complete.groupby("Type")

In [29]:
# Average Reading and Math Score
school_summary_type = pd.DataFrame (school_data_by_type["Math Score", "Reading Score"].mean())
school_summary_type = school_summary_type.rename(columns={"Math Score":"Average Math Score", 
                                                        "Reading Score":"Average Reading Score"})

school_summary_type["% Passing Math"] = 100*(school_data_complete
                                    .loc[school_data_complete["Math Score"] >= 70]
                                    .groupby("Type")["Math Score"].count()
                                    /school_data_by_type["Math Score"].count()
                                   )

# Percentage passing Math and Reading
school_summary_type["% Passing Reading"] = 100*(school_data_complete
                                    .loc[school_data_complete["Reading Score"] >= 70]
                                    .groupby("Type")["Reading Score"].count()
                                    /school_data_by_type["Reading Score"].count()
                                   )


# Overall Passing Rate
school_summary_type["% Overall Passing Rate"] = (school_summary_type["% Passing Math"] 
                                                     + school_summary_type["% Passing Reading"])/2

school_summary_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818


***

### Some Observations

 * Counterintuitively, a larger School Spending per Student, does not mean a better student performance. On the contrary, schools with the least spending (<\\$600/Student) consistently outperformed all other Spending Ranges, including the highest (>\\$650/Student).
 * Similarly, smaller schools consistently outperformed medium- and large-sized ones by a significant margin (for example, look at the % Math Passing rate (94 vs 66%).
 * Charter schools performed significantly better in all categories, when compared to district schools. This trend is also observable when looking at the Top-Performing and Bottom-Performing schools. The five schools at the top are Charter Schools, while the five at the bottom are District Schools.


&#9724;